# Manipulación de datos con Pandas

La idea de esta clase es que aprendamos funcionalidades más avanzadas de pandas para la manipulación de datos que vamos a utilizar mucho en la etapa de feature engineering.


## Combinando datasets: concat(), append(), merge() y join()

Algunos de los estudios de datos más interesantes provienen de la combinación de diferentes datasets. Estas operaciones pueden involucrar cualquier cosa, desde una concatenación muy sencilla de dos datasets, hasta joins al estilo de los que se hacen en SQL.






In [1]:
import pandas as pd
import numpy as np

Creamos una función para armar datasets de forma simple:

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


### Concat

Empecemos con la función concat.

Esta función como su nombre lo indica nos sirve para concatenar Series o DataFrames de pandas.

La concatenación se puede hacer horizontal (axis=1) o vertical (axis=0).

Veamos ejemplos:

#### Concat horizontal

In [3]:
df1 = make_df('AB', [0, 1])
df2 = make_df('CD', [0, 1])

In [4]:
df1

,A,B
0,A0,B0
1,A1,B1


In [5]:
df2

,C,D
0,C0,D0
1,C1,D1


Concatenamos horizontalmente los DF:

In [6]:
pd.concat([df1, df2], axis=1)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


#### Concat vertical

En este caso tenemos que tener en cuenta que los indices se preservan al hacer concat sin importar que nos queden inices repetidos.

In [7]:
df3 = make_df('AB', [0, 1])
df4 = make_df('AB', [2, 3])

# Duplicamos los indices:
df4.index = df3.index

In [8]:
df3

,A,B
0,A0,B0
1,A1,B1


In [9]:
df4

,A,B
0,A2,B2
1,A3,B3


In [11]:
pd.concat([df3, df4], axis=0, ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


¿ Ven algún problema con el concat de arriba ?

Investigar los parámetros:

- verify_integrity
- ignore_index

Y solucionar el problema

In [ ]:
#COMPLETAR

En los ejemplos que estuvimos viendo, los datasets que concatenamos tienen nombres de columnas en común. En la práctica, como los datos vienen de distintas fuentes, van a tener distintas columnas. La función concat de pandas nos ofrece varias opciones para estos casos.

Veamos un ejemplo en el que los dataframes comparten algunas, pero no todas, las columnas:

In [12]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])

In [13]:
df5

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [14]:
df6

,B,C,D
3,B3,C3,D3
4,B4,C4,D4


In [15]:
pd.concat([df5, df6])

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


Vemos que por defecto pandas le asigna un NaN a los valores faltantes. Si no queremos que suceda esto, podemos especificar otras opciones para el "join" (al estilo de lo que se hace en SQL).

Por defecto, el join que se hace es un "outer join", pero podemos cambiar esto a un "inner join"

![image.png](https://estradawebgroup.com/ImagesUpload/sql-joins.jpg)

In [16]:
pd.concat([df5, df6], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


### Append

Este método funciona de manera muy similar a join, nada más que en lugar de tener que hacer ` pd.concat([df1, df2])`, podemos hacer directamente `df1.append(df2)`.

La ventaja es que nos podemos ahorrar algo de código, pero cuando tenemos que concatenar muchos datasets, es más eficiente el método concat.

In [17]:
df3.append(df4)

AttributeError: 'DataFrame' object has no attribute 'append'

### Merge

Uno de los métodos que más vamos a usar a la hora de combinar datasets.

Vimos en la teoría, que esta función implementa:
- one to one
- many to one
- many to many

Veamos ejemplos:

#### One to one

In [18]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [19]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [20]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Si queremos combiar todo esto en un único dataset con un merge:

In [21]:
df3 = pd.merge(df1, df2)
df3


,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Pandas automáticamente reconoce que la columna employee está en ambos dataframes y la utiliza como "key.

#### Many to one

In [22]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

In [23]:
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [24]:
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


En este caso, vemos que en el dataset df3 la columna group tiene el valor Engineering duplicado, por lo tanto esto hace "match" con más de una fila:

In [25]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


En lugar de hacer pd.merge([lista de dfs]) también podemos hacer simplemente df.merge(df2) asi:

In [26]:
df3.merge(df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


#### Many to many

En este caso, hay duplicados en ambos datasets:

In [27]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})

In [28]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [29]:
df5

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [30]:
df1.merge(df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


La función merge, nos deja especificar con que columna queremos hacer el join. Por defecto, vimos que automáticamente reconoce que columna coincide en ambos dataframes y hace el merge.

Si queremos especificarle la columa por la cuál hacer el merge, utilizamos el parámetro "on"

In [31]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Sin embargo, esto únicamente funciona si ambos dataframes tienen una columna que se llama "employee". En el caso de que la columna en ambos dataframes tengan nombres distintos, podemos especificar: "left_on" y "right_on"

In [32]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

In [33]:
df3.head()

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [34]:
df1.head()

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


Aca vemos que en df3 la columna se llama "name" y en df1 la columna se llama "employee". Por lo tanto lo especificamos:

In [35]:
pd.merge(df1, df3, left_on="employee", right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


Podemos eliminar la columna name, ya que es un duplicado de employee

In [36]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


Muchas veces, el merge lo queremos hacer utilizando los índices como "key". Para estos casos, podemos especificar: "left_index = True" cuando queremos utilizar el index como key en el df de la izquierda y "right_index = True" para el de la derecha:

In [37]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

In [38]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [39]:
df2a

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [ ]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


El parámetro left_index=True indica que se utilizará el índice del DataFrame df1a como clave de combinación para los datos de la izquierda. Esto significa que se buscaran las filas en df1a que tengan el mismo índice en df2a y se combinaran en el resultado.

El parámetro right_index=True indica que se utilizará el índice del DataFrame df2a como clave de combinación para los datos de la derecha. Esto significa que se buscaran las filas en df2a que tengan el mismo índice en df1a y se combinaran en el resultado.

En resumen, el código realiza una combinación de dos DataFrames utilizando sus índices como claves de combinación. El resultado será un nuevo DataFrame que contiene las filas de df1a y df2a que tienen los mismos índices.

Esto también, como vimos en la teoría, podemos hacerlo directamente con el método join():

In [ ]:
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


También podemos utilizar el index en un df y la columna en el otro:

In [ ]:
pd.merge(df1a, df3, left_index=True, right_on='name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


### Parámetro how

Vimos que hay distintos tipos de joins entre dataframes (al estilo SQL).
Principalmente vamos a utilizar:

- inner
- left
- right

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

In [ ]:
df6.head()

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [ ]:
df7.head()

,name,drink
0,Mary,wine
1,Joseph,beer


In [ ]:
pd.merge(df6, df7)

,name,food,drink
0,Mary,bread,wine


Vemos que por defecto, el valor del parámetro "on" de la función merge es = "inner". Esto significa que solo va a hacer match si el valor existe en ambos dataframes (intersección).

In [ ]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Ahora, los left y right joins, hacen el merge sobre el dataframe de la izquierda o derecha según corresponda:

In [ ]:
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [ ]:
pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


# Ejercicios

Vamos a trabajar con datasets que se descargan de:
https://www.kaggle.com/c/instacart-market-basket-analysis/data

En la misma página se encuentra toda la información sobre los datasets.

Descargar:
- Order_products_train
- Orders
- Products
- aisles

1) Investigar: Para qué sirve el parámetro suffixes en la función merge? Armar un ejemplo (no hace falta que sea con datasets reales, pueden crear uno de ejemplo como hicimos hasta ahora en el notebook)

2) Quiero obtener un dataset que tenga por cada orden (order_id) un listado (tipo list: valores entre corchetes separados por coma) de NOMBRES de productos que se vendieron en la orden. TIP: groupby + unique

3) Quiero otro dataset, que por cada NOMBRE de producto, tenga el NOMBRE de los distintos aisles en los que lo puedo encontrar en formato de lista.

4) Por cada producto (id de producto), contar en cuantas ordenes distintas se vendió el mismo (acá no hace falta unir dfs, en uno solo tienen la información)
